In [0]:
import SimpleITK as sitk  # For loading the dataset
import numpy as np  # For data manipulation
from model import build_model  # For creating the model
import matplotlib.pyplot as plt
import pandas as pd
from scipy.ndimage import zoom
import cv2
import keras.backend as K
from keras.utils import Sequence
from keras.callbacks.callbacks import LambdaCallback
import cv2

Using TensorFlow backend.


In [0]:
def read_img(img_path):
    """
    Reads a .nii.gz image and returns as a numpy array.
    """
    return sitk.GetArrayFromImage(sitk.ReadImage(img_path))


def resize(img, shape, mode='constant', orig_shape=(155, 240, 240)):
    """
    Wrapper for scipy.ndimage.zoom suited for MRI images.
    """
    assert len(shape) == 3, "Can not have more than 3 dimensions"
    factors = (
        shape[0]/orig_shape[0],
        shape[1]/orig_shape[1], 
        shape[2]/orig_shape[2]
    )
    
    # Resize to the given shape
    return zoom(img, factors, mode=mode)


def preprocess(img, out_shape=None):
    """
    Preprocess the image.
    Just an example, you can add more preprocessing steps if you wish to.
    """
    if out_shape is not None:
        img = resize(img, out_shape, mode='constant')
    
    # Normalize the image
    mean = img.mean()
    std = img.std()
    return (img - mean) / std


def preprocess_label(img, out_shape=None, mode='nearest'):
    """
    Separates out the 3 labels from the segmentation provided, namely:
    GD-enhancing tumor (ET ? label 4), the peritumoral edema (ED ? label 2))
    and the necrotic and non-enhancing tumor core (NCR/NET ? label 1)
    """
    # print(img.shape)
    # print(np.unique(img))
    ncr = img == 1  # Necrotic and Non-Enhancing Tumor (NCR/NET)
    ed = img == 2  # Peritumoral Edema (ED)
    et = img == 4  # GD-enhancing Tumor (ET)
    
    if out_shape is not None:
        ncr = resize(ncr, out_shape, mode=mode)
        ed = resize(ed, out_shape, mode=mode)
        et = resize(et, out_shape, mode=mode)

    return np.array([ncr, ed, et], dtype=np.uint8)

def dice_coefficient(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(K.abs(y_true_f * y_pred_f), axis=-1)
    return (2. * intersection) / (
        K.sum(K.square(y_true_f), -1) + K.sum(K.square(y_pred_f), -1) + 1e-8)

In [0]:
input_shape = (4, 96, 112, 112)
output_channels = 3
model = build_model(input_shape=input_shape, output_channels=3)
model.load_weights('weights/weights.epoch_100-loss_-0.14610-dice_0.64230-val_dice_0.58464.hdf5')

#validation samples
samples=[{'t1': 'HGG/Brats18_TCIA01_460_1/Brats18_TCIA01_460_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA01_460_1/Brats18_TCIA01_460_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA01_460_1/Brats18_TCIA01_460_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA01_460_1/Brats18_TCIA01_460_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA01_460_1/Brats18_TCIA01_460_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_AMH_1/Brats18_CBICA_AMH_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_AMH_1/Brats18_CBICA_AMH_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_AMH_1/Brats18_CBICA_AMH_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_AMH_1/Brats18_CBICA_AMH_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_AMH_1/Brats18_CBICA_AMH_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_ASV_1/Brats18_CBICA_ASV_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_ASV_1/Brats18_CBICA_ASV_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_ASV_1/Brats18_CBICA_ASV_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_ASV_1/Brats18_CBICA_ASV_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_ASV_1/Brats18_CBICA_ASV_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA02_377_1/Brats18_TCIA02_377_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA02_377_1/Brats18_TCIA02_377_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA02_377_1/Brats18_TCIA02_377_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA02_377_1/Brats18_TCIA02_377_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA02_377_1/Brats18_TCIA02_377_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_ATD_1/Brats18_CBICA_ATD_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_ATD_1/Brats18_CBICA_ATD_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_ATD_1/Brats18_CBICA_ATD_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_ATD_1/Brats18_CBICA_ATD_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_ATD_1/Brats18_CBICA_ATD_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA02_374_1/Brats18_TCIA02_374_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA02_374_1/Brats18_TCIA02_374_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA02_374_1/Brats18_TCIA02_374_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA02_374_1/Brats18_TCIA02_374_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA02_374_1/Brats18_TCIA02_374_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA03_133_1/Brats18_TCIA03_133_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA03_133_1/Brats18_TCIA03_133_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA03_133_1/Brats18_TCIA03_133_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA03_133_1/Brats18_TCIA03_133_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA03_133_1/Brats18_TCIA03_133_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA03_265_1/Brats18_TCIA03_265_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA03_265_1/Brats18_TCIA03_265_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA03_265_1/Brats18_TCIA03_265_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA03_265_1/Brats18_TCIA03_265_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA03_265_1/Brats18_TCIA03_265_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA02_394_1/Brats18_TCIA02_394_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA02_394_1/Brats18_TCIA02_394_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA02_394_1/Brats18_TCIA02_394_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA02_394_1/Brats18_TCIA02_394_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA02_394_1/Brats18_TCIA02_394_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_2013_21_1/Brats18_2013_21_1_t1.nii.gz', 't2': 'HGG/Brats18_2013_21_1/Brats18_2013_21_1_t2.nii.gz', 't1ce': 'HGG/Brats18_2013_21_1/Brats18_2013_21_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_2013_21_1/Brats18_2013_21_1_flair.nii.gz', 'seg': 'HGG/Brats18_2013_21_1/Brats18_2013_21_1_seg.nii.gz'}]

W0305 14:48:10.878890 140453689906944 deprecation.py:506] From /usr/local/lib/python3.7/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
kernel = np.ones((3, 3))
File = []
Binary = []
Gray = []
Processed_Binary = []
Processed_Gray = []

for i in samples:
    File.append(i['t1'].split('/')[1])
    print("File: %s"%File[-1])
    
    data_x = np.empty((1,) + input_shape, dtype=np.float32)
    labels = np.empty((1, output_channels) + input_shape[1:], dtype=np.uint8)
    
    data_x[0] = np.array([preprocess(read_img(i[m]), input_shape[1:]) for m in ['t1', 't2', 't1ce', 'flair']], dtype=np.float32)
    labels[0] = preprocess_label(read_img(i['seg']), input_shape[1:])[None, ...]
    
    preds = model.predict(data_x)[0]
    
    pbinary = [] #predicted binary
    pgray = [] #predicted grayscale
    label_gray = [] #original gray
    label_binary = [] #original binary

    binary = [] #processed binary
    gray = [] #processed grayscale
    original_gray = [] #processed original gray
    original_binary = [] #processed original binary
    for i in range(96):
        pred = preds[:, i,:,:]
        pred = np.transpose(pred, (1, 2, 0))
        pred = np.rint(pred)
        pred = pred*255
        pred_gray = cv2.cvtColor(pred, cv2.COLOR_BGR2GRAY)
        pred_gray = pred_gray.astype(np.uint8)
        pgray.append(pred_gray)
        pbinary.append(np.array(pred_gray>0).astype(int))
        pred_gray_closed = cv2.morphologyEx(pred_gray, cv2.MORPH_CLOSE, kernel, iterations=3)
        gray.append(pred_gray_closed)
        pred_binary = np.array(pred_gray_closed>0).astype(int)
        binary.append(pred_binary)
        orig = labels[0][:, i,:,:]
        orig = np.transpose(orig, (1, 2, 0))
        orig = orig*255
        orig_gray = cv2.cvtColor(orig, cv2.COLOR_BGR2GRAY)
        orig_gray = orig_gray.astype(np.uint8)
        label_gray.append(orig_gray)
        label_binary.append(np.array(orig_gray>0).astype(int))
        orig_gray_closed = cv2.morphologyEx(orig_gray, cv2.MORPH_CLOSE, kernel, iterations=3)
        original_gray.append(orig_gray_closed)
        orig_binary = np.array(orig_gray_closed>0).astype(int)
        original_binary.append(orig_binary)
        
    Binary.append(K.get_value(dice_coefficient(K.constant(np.array(label_binary)), K.constant(np.array(pbinary)))))
    Gray.append(K.get_value(dice_coefficient(K.constant(np.array(label_gray)), K.constant(np.array(pgray)))))
    Processed_Binary.append(K.get_value(dice_coefficient(K.constant(np.array(original_binary)), K.constant(np.array(binary)))))
    Processed_Gray.append(K.get_value(dice_coefficient(K.constant(np.array(original_gray)), K.constant(np.array(gray)))))
    print("Dice score without post processing for binary: %s"%Binary[-1])
    print("Dice score without post processing for gray: %s"%Gray[-1])
    print("Dice score with post processing for binary: %s"%Processed_Binary[-1])
    print("Dice score with post processing for gray: %s\n"%Processed_Gray[-1])

File: Brats18_TCIA01_460_1


W0305 14:48:19.806359 140453689906944 module_wrapper.py:139] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



Dice score without processing for binary: 0.5420551
Dice score without processing for gray: 0.44221228
Dice score without processing for processed binary: 0.8815477
Dice score without processing for processed gray: 0.67411387

File: Brats18_CBICA_AMH_1
Dice score without processing for binary: 0.60972184
Dice score without processing for gray: 0.6168306
Dice score without processing for processed binary: 0.92466986
Dice score without processing for processed gray: 0.9132088

File: Brats18_CBICA_ASV_1
Dice score without processing for binary: 0.611202
Dice score without processing for gray: 0.61653197
Dice score without processing for processed binary: 0.9433235
Dice score without processing for processed gray: 0.9377231

File: Brats18_TCIA02_377_1
Dice score without processing for binary: 0.6236397
Dice score without processing for gray: 0.6269005
Dice score without processing for processed binary: 0.9460473
Dice score without processing for processed gray: 0.9392112

File: Brats18_CBI

In [0]:
df = pd.DataFrame({
    'File': File,
    'Binary': Binary,
    'Gray': Gray,
    'Processed_Binary': Processed_Binary,
    'Processed_Gray': Processed_Gray
})
df.to_csv('validation_accuracy.csv', index=False)